In [216]:
#import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras.layers import AveragePooling2D,Dropout,Flatten,Dense,Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from matplotlib.pyplot import figure
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from imutils import paths
from random import shuffle, choice
from PIL import Image
from random import randint
import argparse
import os
import sys
import tensorflow as tf
import numpy as np
import matplotlib as plt
import random
import glob
import matplotlib.pyplot as plt
import time
import pandas as pd
import cv2
import math

from datetime import datetime
import calendar
d = datetime.utcnow()
unixtime = calendar.timegm(d.utctimetuple())
os.mkdir("EXPERIMENT-"+str(unixtime))
path = "EXPERIMENT-"+str(unixtime)+"/"

# Data Loading (Fake data for Now ) 

In [217]:
# example of loading the mnist dataset
from keras.datasets import mnist
from matplotlib import pyplot
# load dataset
(train_data, train_target), (test_data, test_target) = mnist.load_data()
# summarize loaded dataset
                          
train_data = train_data[0:100]                           
train_target = train_target[0:100]   
test_data = test_data[0:10]   
test_target = test_target[0:10]   

print('Train: X=%s, y=%s' % (train_data.shape, train_target.shape))
print('Test: X=%s, y=%s' % (test_data.shape, test_target.shape))
  
#Make 0-5 a group as 0  and 5+ as 1 for simplicity
test_target = np.where(test_target < 5, 0, 1)  
train_target = np.where(train_target < 5, 0, 1)  

train_data = np.array(train_data.reshape((train_data.shape[0], 28, 28, 1)))
test_data = np.array(test_data.reshape((test_data.shape[0], 28, 28, 1)))

train_data =(train_data.repeat(3, -1))  # repeat the last (-1) dimension three times
test_data = (test_data.repeat(3, -1))  # repeat the last (-1) dimension three times

train_data =(train_data.repeat(8, -2))  # repeat the last (-1) dimension three times
test_data = (test_data.repeat(8, -2))  # repeat the last (-1) dimension three times


train_data =(train_data.repeat(8, -3))  # repeat the last (-1) dimension three times
test_data = (test_data.repeat(8, -3))  # repeat the last (-1) dimension three times

tuple(train_target).count(1)
tuple(train_target).count(0)


tuple(test_target).count(1)
tuple(test_target).count(0)
test_data.shape

Train: X=(100, 28, 28), y=(100,)
Test: X=(10, 28, 28), y=(10,)


(10, 224, 224, 3)

In [218]:
def RunModelDesign(settings):
    
  
    #Load the VGG19 model, top layers off
    baseModel = VGG19(weights="imagenet",include_top=False,input_tensor=Input(shape=(224,224,3)))
    INIT_LR= 1e-2 # 0.001 
    EPOCHS= 10
    BS= 8
    
    print("Running Experiment : ",settings)
    print("layer-1:",settings[0])
    headModel=baseModel.output
    headModel=AveragePooling2D(pool_size=(4,4))(headModel)
    headModel=Flatten(name="flatten")(headModel)
    headModel=Dense(settings[0],activation="relu")(headModel) #LAYER 
    headModel=Dense(settings[1],activation="relu")(headModel) #LAYER 
    headModel=Dense(settings[2],activation="relu")(headModel) #LAYER 
    headModel=Dense(settings[3],activation="relu")(headModel) #LAYER 
    headModel=Dropout(settings[4])(headModel)# MIN-MAX 0.1-0.5
    headModel=Dense(2,activation="softmax")(headModel) # abnormal /non compliant 
    model= Model(inputs=baseModel.input, outputs=headModel)
    
    for layer in baseModel.layers:
        layer.trainable = False
        
    if(debug):
        model.summary()
        
    

    print("[INFO] compiling model....")
    opt=Adam(lr=INIT_LR,decay=INIT_LR/EPOCHS)
    model.compile(loss="binary_crossentropy",optimizer=opt,metrics=["accuracy"])
    
    print("[INFO] Training the model....")
    
    
    filepath = path+str(int(settings[0]))+"-"+str(int(settings[1]))+"-"+str(int(settings[2]))+"-"+str(int(settings[3]))+"-DP0"+str(int(10*settings[4]))+"-VGG19.h5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, 
                                 save_best_only=True, mode='max')
    print("[INFO] model file: ",filepath)
    
    
    #initialize training data augmentation 
    trainAug= ImageDataGenerator(rotation_range=15,
                                 rescale=1./255,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 shear_range=0.2,
                                 zoom_range=0.2,
                                 horizontal_flip=True,
                                 fill_mode="nearest")
    

    History=model.fit(
        trainAug.flow(train_data,train_target,batch_size=BS),
        steps_per_epoch=len(train_data)//BS,
        validation_data=(test_data,test_target),
        validation_steps=len(test_data)//BS,
        epochs=1,
        callbacks=[checkpoint])
    
    num_val_samples = len(test_data)
    val_batch_size = BS

    val_steps = np.ceil(num_val_samples / val_batch_size) #160 / 8 = 20 steps per epoch
    print(len(test_data))

        # Here the best epoch will be used.
    #validate the model
    print("[INFO] evaluating model....")
    model.save(filepath)
    model.load_weights(filepath)
    val_loss, val_acc = model.evaluate(test_data,test_target)
    print(test_target)
    print('val_loss:', val_loss)
    print('val_acc:', val_acc)
    
    
    results = [val_loss,val_acc]
    
    return results

## Testrun 1 Population

results = RunModelDesign(popdata[i])
print("Validation Loss ",results[0])
print("Validation Acc ",results[1])

## PSO LOOP

In [219]:

np.set_printoptions(suppress=True)

debug = False
#Initialize Population  [64, 32, 16, 16, 0.1, P, V, W, loss,accuracy , fitness]
popSize = 5
for i in range(popSize):
    
    dropout = round(random.uniform(0.1,0.5), 1)
    layer1 = random.randint(1, 8)*8
    layer2 = random.randint(1, 8)*8
    layer3 = random.randint(1, 8)*8
    layer4 = random.randint(1, 8)*8
    position = round(random.uniform(0,1), 2)
    velocity = round(random.uniform(0,1), 2)
    weight= 0.90
    
    if(debug):
        print("New Population : ",i+1)
       
        print("layer1: ",layer1, end =" ")
        print("layer2: ",layer2, end =" ")
        print("layer3: ",layer3, end =" ")
        print("layer4: ",layer4, end =" ")
        print("dropout: ",dropout, end =" ")
        print("velocity: ",velocity, end =" ")
        print("position: ",position, end =" ")
        print("weight constant: ",weight)
    
    new = [layer1, layer2, layer3,layer4,dropout,position,velocity,weight,1,0,0]
    if(i==0):
        popdata = [new]
    else:
        popdata = np.vstack((popdata, new))
    
    
print("Popualtion Shape : ",popdata.shape)
print(popdata[1])
print("Results Acc:",popdata[1,9])
print("Results Loss:",popdata[1,8])

from datetime import datetime
import calendar
d = datetime.utcnow()
unixtime = calendar.timegm(d.utctimetuple())
#SAVE TO CSV RESULTS

csvFileName = path+"PopualtionData_"+str(popSize)+"_TIME"+str(unixtime)+".csv"
print("Population Results Saved : ",csvFileName )
np.savetxt(csvFileName, popdata, delimiter=",", fmt='%f')


# Update results in tensorflow
print (popdata)

for i in range(popdata.shape[0]):
    
    #Runn The Model Here RANDOM NUMBERS FOR NOW
    accuracy = round(random.uniform(0.5,0.99), 2)
    loss = round(random.uniform(1,0.01), 2)
    # get the results
    results = RunModelDesign(popdata[i]);
    print("Validation Loss ",results[0])
    print("Validation Acc ",results[1])
   
    popdata[i,8]= round(results[0],2)
    popdata[i,9]= round(results[1],2)
    print(popdata)
    print("Saved Progress",popdata[i])
    np.savetxt(csvFileName, popdata, delimiter=",", fmt='%f')
    if(debug):
        print("Runnning Model: ",i+1, " accuracy: ",accuracy," loss: ",loss);
    
print(popdata)








# calculate fitness 

for i in range(popdata.shape[0]):
    accuracy = popdata[i,9]
    loss = popdata[i,8]
    fitness = accuracy + (1-loss)*5
    popdata[i,10]= round(fitness,2)
    print(i+1,"-Calculate Fitness :",round(fitness,2))
    
    


#Find Best model in batch

max_index_col = np.argmax(popdata[:,10], axis=0)
print("Best Model Found", popdata[max_index_col])
global_bestP = popdata[max_index_col,6]
print(" L1     L2     L3    L4    DP    V      P     W    Loss   Acc   Fit")
print(popdata)
# calculate new velocity
#for each denseLAYER COMBINATION 
    # V-new = WEIGHT * V-old + C1 * ( RANDOM(0-1) * P) + (RANDOM(0-1) * C2) dense[i,5] (global best P) - Pold 
    # p-new = P-old + Vnew
    #[00, 00,  00, 0.1,Pnew,Vnew,W,loss,accuracy]

    
    
for i in range(popdata.shape[0]):
    oldV = popdata[i,5]
    oldP = popdata[i,6]  
    weight = popdata[i,7]
    rand01 = round(random.uniform(0,1), 1)
    c1 = 0.1
    c2 = 0.1
    
    newV = weight * oldV +  c1 * ( rand01 * global_bestP ) + (rand01 * c2) - oldP
    popdata[i,5] = round(newV,2)
    newP = oldP + newV
    popdata[i,6] = round(newP,2)
print("Updates New Position adn Velocity")
print(" L1     L2     L3    L4    DP    V      P     W    Loss   Acc   Fit")
print(popdata)
#each model(on an specific epoch) produces val_loss: 0.6911 min - val_accuracy: 0.5063 max
# V-new = WEIGHT * V-old + C1 * RANDOM(0-?)


csvFileName = path+"Updated_PopualtionData_"+str(popSize)+"_TIME"+str(unixtime)+".csv"
print("Population Results Saved : ",csvFileName )
np.savetxt(csvFileName, popdata, delimiter=",", fmt='%f')


Popualtion Shape :  (5, 11)
[24.   48.   24.   48.    0.4   0.2   0.82  0.9   1.    0.    0.  ]
Results Acc: 0.0
Results Loss: 1.0
Population Results Saved :  EXPERIMENT-1616702854/PopualtionData_5_TIME1616702856.csv
[[32.   24.   40.   24.    0.4   0.63  0.06  0.9   1.    0.    0.  ]
 [24.   48.   24.   48.    0.4   0.2   0.82  0.9   1.    0.    0.  ]
 [16.    8.   56.   48.    0.4   0.36  0.87  0.9   1.    0.    0.  ]
 [16.   48.   24.   40.    0.2   0.37  0.74  0.9   1.    0.    0.  ]
 [56.   24.   64.   16.    0.2   0.62  0.15  0.9   1.    0.    0.  ]]
Running Experiment :  [32.   24.   40.   24.    0.4   0.63  0.06  0.9   1.    0.    0.  ]
layer-1: 32.0
[INFO] compiling model....
[INFO] Training the model....
[INFO] model file:  EXPERIMENT-1616702854/32-24-40-24-DP04-VGG19.h5
12/12 [==============================] - ETA: 0s - loss: 0.6961 - accuracy: 0.5000
Epoch 00001: val_accuracy improved from -inf to 0.60000, saving model to EXPERIMENT-1616702854/32-24-40-24-DP04-VGG19.h5
12/1